# Spatial Joins

Avoid cartesian explosion 
~ 90k hotels × 200k OSM POIs x Google maps 20k POI = Too much

## Create GEOHASH function

In [0]:
from pyspark.sql.types import StringType, ArrayType
from pyspark.sql.functions import udf, radians, sin, cos, sqrt, atan2
# Geohash encoding function with type safety
def encode_geohash(lat, lon, precision=6):
    """Encode lat/lon to geohash string"""
    if lat is None or lon is None:
        return None
    
    try:
        lat = float(lat)
        lon = float(lon)
    except (ValueError, TypeError):
        return None
    
    if lat < -90 or lat > 90 or lon < -180 or lon > 180:
        return None
    
    BASE32 = '0123456789bcdefghjkmnpqrstuvwxyz'
    lat_range = [-90.0, 90.0]
    lon_range = [-180.0, 180.0]
    
    geohash = []
    bits = [16, 8, 4, 2, 1]
    bit = 0
    ch = 0
    even = True
    
    while len(geohash) < precision:
        if even:
            mid = (lon_range[0] + lon_range[1]) / 2
            if lon > mid:
                ch |= bits[bit]
                lon_range[0] = mid
            else:
                lon_range[1] = mid
        else:
            mid = (lat_range[0] + lat_range[1]) / 2
            if lat > mid:
                ch |= bits[bit]
                lat_range[0] = mid
            else:
                lat_range[1] = mid
        
        even = not even
        if bit < 4:
            bit += 1
        else:
            geohash.append(BASE32[ch])
            bit = 0
            ch = 0
    
    return ''.join(geohash)


def decode_geohash(geohash):
    """Decode geohash to lat/lon bounds and center"""
    if geohash is None or len(geohash) == 0:
        return None
    
    BASE32 = '0123456789bcdefghjkmnpqrstuvwxyz'
    BASE32_MAP = {c: i for i, c in enumerate(BASE32)}
    
    lat_range = [-90.0, 90.0]
    lon_range = [-180.0, 180.0]
    even = True
    
    for char in geohash:
        idx = BASE32_MAP.get(char, -1)
        if idx < 0:
            return None
        
        for bit in [16, 8, 4, 2, 1]:
            if even:
                mid = (lon_range[0] + lon_range[1]) / 2
                if idx & bit:
                    lon_range[0] = mid
                else:
                    lon_range[1] = mid
            else:
                mid = (lat_range[0] + lat_range[1]) / 2
                if idx & bit:
                    lat_range[0] = mid
                else:
                    lat_range[1] = mid
            even = not even
    
    lat_center = (lat_range[0] + lat_range[1]) / 2
    lon_center = (lon_range[0] + lon_range[1]) / 2
    lat_err = (lat_range[1] - lat_range[0]) / 2
    lon_err = (lon_range[1] - lon_range[0]) / 2
    
    return (lat_center, lon_center, lat_err, lon_err)


def get_geohash_neighbors(geohash):
    """Get a geohash and its 8 spatial neighbors"""
    if geohash is None or len(geohash) == 0:
        return []
    
    decoded = decode_geohash(geohash)
    if decoded is None:
        return [geohash]
    
    lat_center, lon_center, lat_err, lon_err = decoded
    
    # Step size to reach neighboring cells
    lat_step = lat_err * 2
    lon_step = lon_err * 2
    
    neighbors = set()
    neighbors.add(geohash)  # Include self
    
    # 8 directions: N, S, E, W, NE, NW, SE, SW
    directions = [
        (lat_step, 0),           # N
        (-lat_step, 0),          # S
        (0, lon_step),           # E
        (0, -lon_step),          # W
        (lat_step, lon_step),    # NE
        (lat_step, -lon_step),   # NW
        (-lat_step, lon_step),   # SE
        (-lat_step, -lon_step),  # SW
    ]
    
    precision = len(geohash)
    
    for dlat, dlon in directions:
        new_lat = lat_center + dlat
        new_lon = lon_center + dlon
        
        # Handle wraparound
        if new_lat > 90:
            new_lat = 90
        if new_lat < -90:
            new_lat = -90
        if new_lon > 180:
            new_lon -= 360
        if new_lon < -180:
            new_lon += 360
        
        neighbor_hash = encode_geohash(new_lat, new_lon, precision)
        if neighbor_hash:
            neighbors.add(neighbor_hash)
    
    return list(neighbors)


# Register UDFs
geohash_udf = udf(encode_geohash, StringType())
neighbors_udf = udf(get_geohash_neighbors, ArrayType(StringType()))

# Haversine using Spark SQL functions (no UDF needed)
def haversine_spark(lat1, lon1, lat2, lon2):
    """Haversine using Spark SQL functions"""
    R = 6371000  # meters
    
    lat1_rad = radians(lat1)
    lat2_rad = radians(lat2)
    delta_lat = radians(lat2 - lat1)
    delta_lon = radians(lon2 - lon1)
    
    a = sin(delta_lat/2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(delta_lon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    
    return R * c

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, broadcast, radians, sin, cos, sqrt, atan2
from pyspark.sql.functions import sum as spark_sum, avg as spark_avg, min as spark_min, count as spark_count

# Load clean data
booking_clean = spark.read.parquet("dbfs:/FileStore/project/silver/booking_parsed")
osm_clean = spark.read.parquet("dbfs:/FileStore/project/silver/osm_clean")
gmaps_clean = spark.read.parquet("dbfs:/FileStore/project/silver/gmaps_clean")

# Ensure lat/lon are doubles
booking_clean = booking_clean.withColumn("lat", col("lat").cast("double")).withColumn("lon", col("lon").cast("double"))
osm_clean = osm_clean.withColumn("lat", col("lat").cast("double")).withColumn("lon", col("lon").cast("double"))
gmaps_clean = gmaps_clean.withColumn("lat", col("lat").cast("double")).withColumn("lon", col("lon").cast("double"))

# Filter nulls
booking_clean = booking_clean.filter(col("lat").isNotNull() & col("lon").isNotNull())
osm_clean = osm_clean.filter(col("lat").isNotNull() & col("lon").isNotNull())
gmaps_clean = gmaps_clean.filter(col("lat").isNotNull() & col("lon").isNotNull())

print(f"Hotels: {booking_clean.count():,}")
print(f"OSM POIs: {osm_clean.count():,}")
print(f"GMaps POIs: {gmaps_clean.count():,}")

Hotels: 62,643
OSM POIs: 162,984
GMaps POIs: 10,807


In [0]:
# Haversine distance function
def haversine_spark(lat1, lon1, lat2, lon2):
    R = 6371000  # meters
    lat1_rad = radians(lat1)
    lat2_rad = radians(lat2)
    delta_lat = radians(lat2 - lat1)
    delta_lon = radians(lon2 - lon1)
    a = sin(delta_lat/2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(delta_lon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

# Process each city separately (more manageable)
CITIES = ["Amsterdam", "Bangkok", "Dubai", "Eilat", "Haifa", "London", "New York", "Rome", "Tel Aviv", "Tokyo"]
BBOX_DEGREES = 0.015  # ~1.5km

all_results = []

for city in CITIES:
    print(f"\nProcessing {city}...")
    
    # Filter to city
    hotels_city = booking_clean.filter(col("city") == city).select(
        col("hotel_id"),
        col("lat").alias("hotel_lat"),
        col("lon").alias("hotel_lon")
    )
    
    osm_city = osm_clean.filter(col("city") == city).select(
        col("lat").alias("poi_lat"),
        col("lon").alias("poi_lon"),
        col("poi_category"),
        col("poi_group")
    )
    
    hotel_count = hotels_city.count()
    poi_count = osm_city.count()
    print(f"  Hotels: {hotel_count}, POIs: {poi_count}")
    
    if hotel_count == 0 or poi_count == 0:
        continue
    
    # Broadcast smaller DataFrame (POIs are usually smaller per city)
    if poi_count < 50000:
        osm_city = broadcast(osm_city)
    
    # Cross join with bounding box
    joined = hotels_city.crossJoin(osm_city).filter(
        (col("poi_lat").between(col("hotel_lat") - BBOX_DEGREES, col("hotel_lat") + BBOX_DEGREES)) &
        (col("poi_lon").between(col("hotel_lon") - BBOX_DEGREES, col("hotel_lon") + BBOX_DEGREES))
    )
    
    # Calculate distance
    joined = joined.withColumn(
        "distance_m",
        haversine_spark(col("hotel_lat"), col("hotel_lon"), col("poi_lat"), col("poi_lon"))
    )
    
    # Filter to 1km
    joined = joined.filter(col("distance_m") <= 1000)
    
    print(f"  Pairs within 1km: {joined.count():,}")
    
    # Aggregate features per hotel
    city_features = joined.groupBy("hotel_id").agg(
        # Noise sources
        spark_count(when((col("poi_group") == "noise_source") & (col("distance_m") <= 300), 1)).alias("noise_sources_300m"),
        spark_count(when((col("poi_group") == "noise_source") & (col("distance_m") <= 500), 1)).alias("noise_sources_500m"),
        spark_count(when((col("poi_category") == "nightlife") & (col("distance_m") <= 500), 1)).alias("nightlife_count_500m"),
        
        # Transport
        spark_count(when((col("poi_group") == "transport") & (col("distance_m") <= 500), 1)).alias("transport_stops_500m"),
        spark_count(when((col("poi_category") == "transit_station") & (col("distance_m") <= 1000), 1)).alias("transit_stations_1km"),
        spark_min(when(col("poi_group") == "transport", col("distance_m"))).alias("nearest_transport_m"),
        
        # Dining
        spark_count(when((col("poi_group") == "dining") & (col("distance_m") <= 300), 1)).alias("restaurants_300m"),
        spark_count(when((col("poi_group") == "dining") & (col("distance_m") <= 500), 1)).alias("restaurants_500m"),
        
        # Leisure
        spark_count(when((col("poi_group") == "leisure") & (col("distance_m") <= 500), 1)).alias("leisure_500m"),
        spark_count(when((col("poi_category") == "park") & (col("distance_m") <= 500), 1)).alias("parks_500m"),
        spark_min(when(col("poi_category") == "park", col("distance_m"))).alias("nearest_park_m"),
        
        # Tourism
        spark_count(when((col("poi_group") == "tourism") & (col("distance_m") <= 1000), 1)).alias("tourism_pois_1km"),
        spark_min(when(col("poi_group") == "tourism", col("distance_m"))).alias("nearest_attraction_m"),
        
        # Convenience
        spark_count(when((col("poi_group") == "convenience") & (col("distance_m") <= 500), 1)).alias("convenience_500m"),
        
        # Total
        spark_count("*").alias("total_osm_pois_1km")
    )
    
    all_results.append(city_features)
    print(f"  Done!")

# Union all cities
osm_features = all_results[0]
for df in all_results[1:]:
    osm_features = osm_features.union(df)

print(f"\nTotal hotels with OSM features: {osm_features.count():,}")


Processing Amsterdam...
  Hotels: 985, POIs: 14426
  Pairs within 1km: 828,902
  Done!

Processing Bangkok...
  Hotels: 2953, POIs: 19686
  Pairs within 1km: 1,159,650
  Done!

Processing Dubai...
  Hotels: 15250, POIs: 1436
  Pairs within 1km: 441,798
  Done!

Processing Eilat...
  Hotels: 680, POIs: 795
  Pairs within 1km: 115,043
  Done!

Processing Haifa...
  Hotels: 262, POIs: 2022
  Pairs within 1km: 41,094
  Done!

Processing London...
  Hotels: 15556, POIs: 35387
  Pairs within 1km: 9,007,708
  Done!

Processing New York...
  Hotels: 2255, POIs: 26767
  Pairs within 1km: 2,246,230
  Done!

Processing Rome...
  Hotels: 16883, POIs: 23851
  Pairs within 1km: 10,842,992
  Done!

Processing Tel Aviv...
  Hotels: 955, POIs: 3644
  Pairs within 1km: 390,110
  Done!

Processing Tokyo...
  Hotels: 6864, POIs: 34970
  Pairs within 1km: 4,150,103
  Done!

Total hotels with OSM features: 50,029


## Load all datasets

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, udf, explode, array, lit, when
from pyspark.sql.functions import sum as spark_sum, avg as spark_avg, min as spark_min, count as spark_count
from pyspark.sql.types import StringType, ArrayType, DoubleType
from pyspark.sql.functions import radians, sin, cos, sqrt, atan2
import math

# Load cleaned datasets
print("Loading datasets...")

booking_clean = spark.read.parquet("dbfs:/FileStore/project/silver/booking_parsed")
osm_clean = spark.read.parquet("dbfs:/FileStore/project/silver/osm_clean")
gmaps_clean = spark.read.parquet("dbfs:/FileStore/project/silver/gmaps_clean")

# Check and fix data types
print("\n=== Checking lat/lon types ===")
print(f"Booking lat type: {dict(booking_clean.dtypes).get('lat')}")
print(f"OSM lat type: {dict(osm_clean.dtypes).get('lat')}")
print(f"GMaps lat type: {dict(gmaps_clean.dtypes).get('lat')}")

# Cast all lat/lon to double
booking_clean = booking_clean.withColumn(
    "lat", col("lat").cast("double")
).withColumn(
    "lon", col("lon").cast("double")
)

osm_clean = osm_clean.withColumn(
    "lat", col("lat").cast("double")
).withColumn(
    "lon", col("lon").cast("double")
)

gmaps_clean = gmaps_clean.withColumn(
    "lat", col("lat").cast("double")
).withColumn(
    "lon", col("lon").cast("double")
)

# Filter out nulls
booking_clean = booking_clean.filter(col("lat").isNotNull() & col("lon").isNotNull())
osm_clean = osm_clean.filter(col("lat").isNotNull() & col("lon").isNotNull())
gmaps_clean = gmaps_clean.filter(col("lat").isNotNull() & col("lon").isNotNull())

print(f"\nHotels: {booking_clean.count():,}")
print(f"OSM POIs: {osm_clean.count():,}")
print(f"Google Maps POIs: {gmaps_clean.count():,}")

Loading datasets...

=== Checking lat/lon types ===
Booking lat type: double
OSM lat type: string
GMaps lat type: double

Hotels: 62,643
OSM POIs: 162,984
Google Maps POIs: 10,807


In [0]:
# Verify cities match across datasets
print("\n=== Hotels per City ===")
display(booking_clean.groupBy("city").count().orderBy("city"))

print("\n=== OSM POIs per City ===")
display(osm_clean.groupBy("city").count().orderBy("city"))

print("\n=== Google Maps POIs per City ===")
display(gmaps_clean.groupBy("city").count().orderBy("city"))


=== Hotels per City ===


city,count
Amsterdam,985
Bangkok,2953
Dubai,15250
Eilat,680
Haifa,262
London,15556
New York,2255
Rome,16883
Tel Aviv,955
Tokyo,6864



=== OSM POIs per City ===


city,count
Amsterdam,14426
Bangkok,19686
Dubai,1436
Eilat,795
Haifa,2022
London,35387
New York,26767
Rome,23851
Tel Aviv,3644
Tokyo,34970



=== Google Maps POIs per City ===


city,count
Amsterdam,1708
Bangkok,2317
Dubai,1924
Eilat,1461
Rome,1678
Tokyo,1719


## Add geo-Hash

In [0]:
# Add geohash to hotels
print("Adding geohash to hotels...")
booking_with_geohash = booking_clean.withColumn(
    "geohash",
    geohash_udf(col("lat"), col("lon"))
).withColumn(
    "geohash_neighbors",
    neighbors_udf(col("geohash"))
).filter(
    col("geohash").isNotNull()
)

print(f"Hotels with geohash: {booking_with_geohash.count():,}")

# Add geohash to OSM
print("Adding geohash to OSM...")
osm_with_geohash = osm_clean.withColumn(
    "geohash",
    geohash_udf(col("lat"), col("lon"))
).filter(
    col("geohash").isNotNull()
)

print(f"OSM POIs with geohash: {osm_with_geohash.count():,}")

# Add geohash to Google Maps
print("Adding geohash to Google Maps...")
gmaps_with_geohash = gmaps_clean.withColumn(
    "geohash",
    geohash_udf(col("lat"), col("lon"))
).filter(
    col("geohash").isNotNull()
)

print(f"GMaps POIs with geohash: {gmaps_with_geohash.count():,}")

Adding geohash to hotels...
Hotels with geohash: 62,643
Adding geohash to OSM...
OSM POIs with geohash: 162,984
Adding geohash to Google Maps...
GMaps POIs with geohash: 10,807


In [0]:
# Verify geohashes look correct
print("=== Sample Geohashes ===")
display(
    booking_with_geohash
    .select("hotel_id", "city", "lat", "lon", "geohash")
    .limit(10)
)

=== Sample Geohashes ===


hotel_id,city,lat,lon,geohash
10836680,Rome,41.8882287,12.5001644,sr2yk3
1246084,Rome,41.8966138,12.478753099999949,sr2y7g
81972,Rome,41.903363034132724,12.50720500946045,sr2yks
2789192,Rome,41.9017662,12.4853937,sr2yk5
80245,Rome,41.90436516585404,12.499953657388687,sr2ykk
2196276,Rome,41.8982260459089,12.4695482963299,sr2y7g
9480889,Rome,41.900865201606,12.534334261377,sr2ym5
9293517,Rome,41.8980378,12.5034979,sr2yke
1181419,Rome,41.9094926,12.461840100000018,sr2y7t
9494936,Rome,41.9137469,12.5278058,sr2ymn


## Spatial Join with OSM

In [0]:
# Explode hotel geohash neighbors for join
hotels_exploded = booking_with_geohash.select(
    col("hotel_id"),
    col("city").alias("hotel_city"),
    col("lat").alias("hotel_lat"),
    col("lon").alias("hotel_lon"),
    explode(col("geohash_neighbors")).alias("join_geohash")
)

print(f"Hotels exploded rows: {hotels_exploded.count():,}")

# Join hotels with OSM POIs on geohash
hotel_osm_joined = hotels_exploded.join(
    osm_with_geohash.select(
        col("city").alias("poi_city"),
        col("lat").alias("poi_lat"),
        col("lon").alias("poi_lon"),
        col("poi_category"),
        col("poi_group"),
        col("name").alias("poi_name"),
        col("geohash").alias("poi_geohash")
    ),
    (col("join_geohash") == col("poi_geohash")) & 
    (col("hotel_city") == col("poi_city")),
    "inner"
)

print(f"Hotel-OSM joined pairs: {hotel_osm_joined.count():,}")

Hotels exploded rows: 563,787
Hotel-OSM joined pairs: 42,218,354


In [0]:
# Calculate actual distance
hotel_osm_with_distance = hotel_osm_joined.withColumn(
    "distance_m",
    haversine_spark(col("hotel_lat"), col("hotel_lon"), col("poi_lat"), col("poi_lon"))
)

# Filter to relevant distances (within 1km)
hotel_osm_nearby = hotel_osm_with_distance.filter(col("distance_m") <= 1000)

print(f"Hotel-OSM pairs within 1km: {hotel_osm_nearby.count():,}")

Hotel-OSM pairs within 1km: 27,102,574


In [0]:
# Check sample joins
display(
    hotel_osm_nearby
    .select("hotel_id", "hotel_city", "poi_name", "poi_category", "poi_group", "distance_m")
    .orderBy("hotel_id", "distance_m")
    .limit(20)
)

hotel_id,hotel_city,poi_name,poi_category,poi_group,distance_m
10000596,London,Cadogan Square Garden,park,leisure,60.6427372586395
10000596,London,Gertrude Bell,historic_site,tourism,88.60675084255745
10000596,London,Arnold Bennett,historic_site,tourism,112.60097506411373
10000596,London,Martha Gellhorn,historic_site,tourism,131.55812460879974
10000596,London,Sussex House School,education,education,134.74674913844828
10000596,London,Cadogan Place South Garden,park,leisure,146.75066446898106
10000596,London,null,park,leisure,157.64778404750825
10000596,London,Hans' Bar & Grill,restaurant,dining,167.35000141703557
10000596,London,Pont Street,transit_stop,transport,168.29887389734756
10000596,London,Herbert Beerbohm Tree,historic_site,tourism,169.58909447281846


In [0]:
# Define distance thresholds
RADIUS_300M = 300
RADIUS_500M = 500
RADIUS_1KM = 1000

# Aggregate OSM features per hotel
osm_features = hotel_osm_nearby.groupBy("hotel_id", "hotel_city").agg(
    # ===== NOISE SOURCES (nightlife) =====
    spark_count(when((col("poi_group") == "noise_source") & (col("distance_m") <= RADIUS_300M), 1)).alias("noise_sources_300m"),
    spark_count(when((col("poi_group") == "noise_source") & (col("distance_m") <= RADIUS_500M), 1)).alias("noise_sources_500m"),
    spark_count(when((col("poi_category") == "nightlife") & (col("distance_m") <= RADIUS_500M), 1)).alias("nightlife_count_500m"),
    
    # ===== TRANSPORT =====
    spark_count(when((col("poi_group") == "transport") & (col("distance_m") <= RADIUS_500M), 1)).alias("transport_stops_500m"),
    spark_count(when((col("poi_category") == "transit_station") & (col("distance_m") <= RADIUS_1KM), 1)).alias("transit_stations_1km"),
    spark_min(when(col("poi_group") == "transport", col("distance_m"))).alias("nearest_transport_m"),
    spark_min(when(col("poi_category") == "transit_station", col("distance_m"))).alias("nearest_station_m"),
    
    # ===== DINING =====
    spark_count(when((col("poi_group") == "dining") & (col("distance_m") <= RADIUS_300M), 1)).alias("restaurants_300m"),
    spark_count(when((col("poi_group") == "dining") & (col("distance_m") <= RADIUS_500M), 1)).alias("restaurants_500m"),
    
    # ===== LEISURE (parks, etc.) =====
    spark_count(when((col("poi_group") == "leisure") & (col("distance_m") <= RADIUS_500M), 1)).alias("leisure_500m"),
    spark_count(when((col("poi_category") == "park") & (col("distance_m") <= RADIUS_500M), 1)).alias("parks_500m"),
    spark_min(when(col("poi_category") == "park", col("distance_m"))).alias("nearest_park_m"),
    
    # ===== TOURISM =====
    spark_count(when((col("poi_group") == "tourism") & (col("distance_m") <= RADIUS_1KM), 1)).alias("tourism_pois_1km"),
    spark_min(when(col("poi_group") == "tourism", col("distance_m"))).alias("nearest_attraction_m"),
    
    # ===== CONVENIENCE =====
    spark_count(when((col("poi_group") == "convenience") & (col("distance_m") <= RADIUS_500M), 1)).alias("convenience_500m"),
    
    # ===== TOTAL COUNT =====
    spark_count("*").alias("total_osm_pois_1km")
)

print(f"Hotels with OSM features: {osm_features.count():,}")
display(osm_features.limit(10))

Hotels with OSM features: 49,936


hotel_id,hotel_city,noise_sources_300m,noise_sources_500m,nightlife_count_500m,transport_stops_500m,transit_stations_1km,nearest_transport_m,nearest_station_m,restaurants_300m,restaurants_500m,leisure_500m,parks_500m,nearest_park_m,tourism_pois_1km,nearest_attraction_m,convenience_500m,total_osm_pois_1km
8734970,Rome,0,0,0,5,0,142.1823165249861,null,1,1,0,0,689.9283692821382,2,663.0350575054069,0,101
81121,Rome,2,5,5,32,9,69.80974255512272,268.32174563162044,40,70,11,11,200.48041959866015,58,211.68827835462756,11,670
323469,Rome,1,5,5,53,9,51.83444905805156,261.9221972869781,56,155,22,22,92.3080075174936,91,97.35900037712705,32,875
6132472,Rome,0,4,4,25,4,153.68899800928895,245.8593311060939,4,25,13,13,350.6439802408431,60,269.74682318684603,11,489
12020943,Rome,4,15,15,23,6,118.71435467188961,200.7228328100873,15,48,8,8,97.20759359550438,31,284.9324662715593,6,390
5612101,Rome,3,3,3,12,0,154.1747504065466,null,5,10,7,7,225.98111305453568,42,247.84537993806575,0,224
14054609,Rome,1,4,4,74,10,166.88002965384595,184.18099272069222,57,127,16,16,331.2241821813683,80,359.78484413242387,15,823
9521869,Rome,1,3,3,19,6,183.819764105866,282.18435137521806,2,15,4,4,185.59317316217067,5,328.8685353162591,3,243
7794891,Rome,4,12,12,24,2,92.34098962054344,416.175961801599,52,212,74,74,205.420427224053,161,111.81359185593557,12,1067
2314224,Rome,15,18,18,14,1,248.978615639661,955.7541686204687,79,144,94,94,63.38327884563009,115,118.83730729220268,8,753


## Spatial Join with Google maps

In [0]:
# Join hotels with Google Maps POIs (same approach)
hotel_gmaps_joined = hotels_exploded.join(
    gmaps_with_geohash.select(
        col("city").alias("poi_city"),
        col("lat").alias("poi_lat"),
        col("lon").alias("poi_lon"),
        col("poi_category"),
        col("poi_group"),
        col("place_name").alias("poi_name"),
        col("rating").alias("poi_rating"),
        col("num_of_reviews").alias("poi_reviews"),
        col("noise_score"),
        col("vibe_tourist_trap"),
        col("vibe_local_favorite"),
        col("vibe_family_friendly"),
        col("vibe_sketchy"),
        col("vibe_romantic"),
        col("geohash").alias("poi_geohash")
    ),
    (col("join_geohash") == col("poi_geohash")) & 
    (col("hotel_city") == col("poi_city")),
    "inner"
)

# Calculate distance
hotel_gmaps_with_distance = hotel_gmaps_joined.withColumn(
    "distance_m",
    haversine_spark(col("hotel_lat"), col("hotel_lon"), col("poi_lat"), col("poi_lon"))
)

# Filter to within 1km
hotel_gmaps_nearby = hotel_gmaps_with_distance.filter(col("distance_m") <= 1000)

print(f"Hotel-GMaps pairs within 1km: {hotel_gmaps_nearby.count():,}")

Hotel-GMaps pairs within 1km: 3,244,893


### Agregate Google map features per hotel

In [0]:
# Aggregate Google Maps features per hotel
gmaps_features = hotel_gmaps_nearby.groupBy("hotel_id", "hotel_city").agg(
    # ===== NIGHTLIFE WITH QUALITY =====
    spark_count(when((col("poi_group") == "nightlife") & (col("distance_m") <= RADIUS_500M), 1)).alias("gmaps_nightlife_500m"),
    spark_avg(when((col("poi_group") == "nightlife") & (col("distance_m") <= RADIUS_500M), col("poi_rating"))).alias("avg_nightlife_rating_500m"),
    spark_avg(when((col("poi_group") == "nightlife") & (col("distance_m") <= RADIUS_500M), col("noise_score"))).alias("avg_noise_score_500m"),
    spark_sum(when((col("poi_group") == "nightlife") & (col("distance_m") <= RADIUS_500M), col("poi_reviews"))).alias("total_nightlife_reviews_500m"),
    
    # ===== DINING WITH QUALITY =====
    spark_count(when((col("poi_group") == "dining") & (col("distance_m") <= RADIUS_300M), 1)).alias("gmaps_restaurants_300m"),
    spark_count(when((col("poi_group") == "dining") & (col("distance_m") <= RADIUS_500M), 1)).alias("gmaps_restaurants_500m"),
    spark_avg(when((col("poi_group") == "dining") & (col("distance_m") <= RADIUS_500M), col("poi_rating"))).alias("avg_restaurant_rating_500m"),
    spark_count(when((col("poi_group") == "dining") & (col("distance_m") <= RADIUS_500M) & (col("poi_rating") >= 4.0), 1)).alias("high_rated_restaurants_500m"),
    
    # ===== TOURISM WITH QUALITY =====
    spark_count(when((col("poi_group") == "tourism") & (col("distance_m") <= RADIUS_1KM), 1)).alias("gmaps_attractions_1km"),
    spark_avg(when((col("poi_group") == "tourism") & (col("distance_m") <= RADIUS_1KM), col("poi_rating"))).alias("avg_attraction_rating_1km"),
    
    # ===== VIBE SCORES =====
    spark_sum(when((col("vibe_tourist_trap") == True) & (col("distance_m") <= RADIUS_500M), 1).otherwise(0)).alias("tourist_trap_pois_500m"),
    spark_sum(when((col("vibe_local_favorite") == True) & (col("distance_m") <= RADIUS_500M), 1).otherwise(0)).alias("local_favorite_pois_500m"),
    spark_sum(when((col("vibe_family_friendly") == True) & (col("distance_m") <= RADIUS_500M), 1).otherwise(0)).alias("family_friendly_pois_500m"),
    spark_sum(when((col("vibe_sketchy") == True) & (col("distance_m") <= RADIUS_500M), 1).otherwise(0)).alias("sketchy_pois_500m"),
    spark_sum(when((col("vibe_romantic") == True) & (col("distance_m") <= RADIUS_500M), 1).otherwise(0)).alias("romantic_pois_500m"),
    
    # ===== OVERALL NEIGHBORHOOD QUALITY =====
    spark_avg(when(col("distance_m") <= RADIUS_500M, col("poi_rating"))).alias("neighborhood_avg_rating_500m"),
    spark_count("*").alias("total_gmaps_pois_1km")
)

print(f"Hotels with GMaps features: {gmaps_features.count():,}")
display(gmaps_features.limit(10))

Hotels with GMaps features: 41,006


hotel_id,hotel_city,gmaps_nightlife_500m,avg_nightlife_rating_500m,avg_noise_score_500m,total_nightlife_reviews_500m,gmaps_restaurants_300m,gmaps_restaurants_500m,avg_restaurant_rating_500m,high_rated_restaurants_500m,gmaps_attractions_1km,avg_attraction_rating_1km,tourist_trap_pois_500m,local_favorite_pois_500m,family_friendly_pois_500m,sketchy_pois_500m,romantic_pois_500m,neighborhood_avg_rating_500m,total_gmaps_pois_1km
8734970,Rome,0,null,null,null,0,0,null,0,3,4.933333333333334,0,0,1,0,1,4.9,5
81121,Rome,3,4.6000000000000005,0.8000000000000002,3400,1,2,4.65,2,18,4.4222222222222225,12,9,9,4,10,4.3238095238095235,72
323469,Rome,5,4.36,0.6166666666666666,17037,4,12,4.458333333333333,11,37,4.581081081081081,44,33,44,3,45,4.5851851851851855,165
6132472,Rome,2,4.15,0.5,4619,1,3,4.566666666666666,3,17,4.558823529411764,9,4,10,2,11,4.49090909090909,89
12020943,Rome,3,4.733333333333333,0.8190476190476191,280,0,1,4.4,1,8,4.1375,1,5,5,2,6,4.56,35
5612101,Rome,0,null,null,null,0,0,null,0,13,4.684615384615385,0,0,0,0,0,4.699999999999999,17
14054609,Rome,3,4.2,0.6944444444444443,9828,1,6,4.116666666666666,5,29,4.5,16,11,18,2,18,4.400000000000001,112
9521869,Rome,1,4.7,0.5,161,0,0,null,0,3,4.433333333333334,1,1,4,1,2,4.52,23
7794891,Rome,17,4.564705882352942,0.5476890756302522,20838,10,17,4.570588235294117,17,67,4.717910447761192,45,42,57,10,55,4.657317073170731,210
2314224,Rome,13,4.461538461538462,0.5878276313058921,16372,2,6,4.633333333333334,6,36,4.66111111111111,31,21,30,10,31,4.577272727272727,111


## Join all data

In [0]:
# Start with booking data
hotel_enriched = booking_clean

# Join OSM features
hotel_enriched = hotel_enriched.join(
    osm_features.drop("hotel_city"),
    on="hotel_id",
    how="left"
)

# Join Google Maps features
hotel_enriched = hotel_enriched.join(
    gmaps_features.drop("hotel_city"),
    on="hotel_id",
    how="left"
)

# Fill nulls with 0 for count features
count_columns = [
    "noise_sources_300m", "noise_sources_500m", "nightlife_count_500m",
    "transport_stops_500m", "transit_stations_1km", "restaurants_300m", "restaurants_500m",
    "leisure_500m", "parks_500m", "tourism_pois_1km", "convenience_500m", "total_osm_pois_1km",
    "gmaps_nightlife_500m", "gmaps_restaurants_300m", "gmaps_restaurants_500m",
    "high_rated_restaurants_500m", "gmaps_attractions_1km",
    "tourist_trap_pois_500m", "local_favorite_pois_500m", "family_friendly_pois_500m",
    "sketchy_pois_500m", "romantic_pois_500m", "total_gmaps_pois_1km"
]

for col_name in count_columns:
    if col_name in hotel_enriched.columns:
        hotel_enriched = hotel_enriched.withColumn(
            col_name,
            when(col(col_name).isNull(), 0).otherwise(col(col_name))
        )

# Cache the result
hotel_enriched.cache()

print(f"Total enriched hotels: {hotel_enriched.count():,}")
hotel_enriched.printSchema()

Total enriched hotels: 62,643
root
 |-- hotel_id: string (nullable = true)
 |-- city_original: string (nullable = true)
 |-- country: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- review_score: double (nullable = true)
 |-- number_of_reviews: long (nullable = true)
 |-- manager_score: double (nullable = true)
 |-- metro_railway_access: boolean (nullable = true)
 |-- all_reviews_text: string (nullable = true)
 |-- num_reviews_available: integer (nullable = true)
 |-- num_reviewer_countries: integer (nullable = true)
 |-- has_wifi: boolean (nullable = true)
 |-- has_parking: boolean (nullable = true)
 |-- has_ac: boolean (nullable = true)
 |-- has_pool: boolean (nullable = true)
 |-- has_gym: boolean (nullable = true)
 |-- has_kitchen: boolean (nullable = true)
 |-- has_breakfast: boolean (nullable = true)
 |-- has_restaurant: boolean (nullable = true)
 |-- has_sp

In [0]:
# Check enrichment summary
print("=== Enrichment Summary by City ===")
display(
    hotel_enriched
    .groupBy("city")
    .agg(
        spark_count("*").alias("hotels"),
        F.round(spark_avg("nightlife_count_500m"), 1).alias("avg_nightlife_500m"),
        F.round(spark_avg("noise_sources_500m"), 1).alias("avg_noise_sources_500m"),
        F.round(spark_avg("restaurants_500m"), 1).alias("avg_restaurants_500m"),
        F.round(spark_avg("nearest_transport_m"), 0).alias("avg_nearest_transport_m"),
        F.round(spark_avg("avg_restaurant_rating_500m"), 2).alias("avg_restaurant_quality"),
        F.round(spark_avg("avg_noise_score_500m"), 2).alias("avg_noise_score")
    )
    .orderBy("city")
)
# Sample enriched hotels
display(
    hotel_enriched
    .select(
        "hotel_id", "city", "title", "review_score",
        "nightlife_count_500m", "noise_sources_500m", "avg_noise_score_500m",
        "restaurants_500m", "avg_restaurant_rating_500m",
        "nearest_transport_m", "tourist_trap_pois_500m"
    )
    .orderBy(F.desc("nightlife_count_500m"))
    .limit(20)
)

=== Enrichment Summary by City ===


city,hotels,avg_nightlife_500m,avg_noise_sources_500m,avg_restaurants_500m,avg_nearest_transport_m,avg_restaurant_quality,avg_noise_score
Amsterdam,985,32.6,32.6,118.1,184.0,4.55,0.58
Bangkok,2953,13.9,13.9,62.2,241.0,4.6,0.5
Dubai,15250,0.5,0.5,5.1,382.0,4.72,0.67
Eilat,680,0.9,0.9,7.3,137.0,4.45,0.63
Haifa,262,2.1,2.1,13.6,92.0,null,null
London,15556,16.8,16.8,73.1,125.0,null,null
New York,2255,21.8,21.8,132.1,101.0,null,null
Rome,16883,7.8,7.8,88.8,100.0,4.57,0.58
Tel Aviv,955,16.2,16.2,61.0,96.0,null,null
Tokyo,6864,30.7,30.7,82.0,167.0,4.4,0.55


hotel_id,city,title,review_score,nightlife_count_500m,noise_sources_500m,avg_noise_score_500m,restaurants_500m,avg_restaurant_rating_500m,nearest_transport_m,tourist_trap_pois_500m
6619499,Tokyo,nestay villa tokyo akihabara,7.5,641,641,0.5715151515151515,728,4.9,42.30199833880349,14
2558061,Tokyo,null,null,625,625,0.6727272727272728,709,null,20.426852980025892,11
13097962,Tokyo,上野公园旁日式民宿3,null,621,621,0.5715151515151515,694,null,40.40507147796423,13
13892878,Tokyo,上野日式民宿,null,621,621,0.5715151515151515,694,null,40.69513507978031,13
778345,Tokyo,Ueno Station Hostel Oriental 1 Male Only,6.9,619,619,0.5715151515151515,649,4.4,141.89647121643105,11
778341,Tokyo,null,null,619,619,0.5715151515151515,650,4.4,142.07748402517376,11
6088709,Tokyo,Tosei Hotel Cocone Ueno Okachimachi,8.5,613,613,0.6727272727272728,701,null,55.903793118412494,11
4210074,Tokyo,null,null,613,613,0.6727272727272728,671,null,59.17911269164578,11
820976,Tokyo,Centurion Hotel Ueno,6.9,610,610,0.6727272727272728,666,null,26.1925655369521,11
8562811,Tokyo,null,null,606,606,0.5715151515151515,659,null,79.05054686572461,12


In [0]:
# Save to Gold layer
hotel_enriched.write \
    .mode("overwrite") \
    .partitionBy("city") \
    .parquet("dbfs:/FileStore/project/gold/hotel_enriched")

print("Saved to: dbfs:/FileStore/project/gold/hotel_enriched")

Saved to: dbfs:/FileStore/project/gold/hotel_enriched


# NLP Pipeline

## First part

we use basic key words matching for this part

### Load Data

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, lower, length, size, split, regexp_extract
from pyspark.sql.functions import sum as spark_sum, avg as spark_avg, count as spark_count

# Load the enriched hotel data
hotel_enriched = spark.read.parquet("dbfs:/FileStore/project/gold/hotel_enriched")

print(f"Total hotels: {hotel_enriched.count():,}")

# Check how many have reviews
reviews_available = hotel_enriched.filter(
    col("all_reviews_text").isNotNull() & 
    (length(col("all_reviews_text")) > 10)
).count()

print(f"Hotels with reviews: {reviews_available:,} ({reviews_available/hotel_enriched.count()*100:.1f}%)")

Total hotels: 62,643
Hotels with reviews: 37,189 (59.4%)


### Key words

In [0]:
# ============================================================
# COMPLAINT KEYWORDS (Reality Signals - Negative)
# ============================================================

COMPLAINT_KEYWORDS = {
    "noise": [
        "noisy", "loud", "noise", "couldn't sleep", "could not sleep",
        "thin walls", "hear everything", "street noise", "traffic noise",
        "party", "music all night", "neighbors", "construction",
        "earplugs", "sleepless", "woke up", "disturbed"
    ],
    "cleanliness": [
        "dirty", "unclean", "filthy", "stains", "stained", "dusty",
        "smell", "smelly", "odor", "mold", "mouldy", "moldy",
        "hair", "bugs", "cockroach", "insects", "ants",
        "not clean", "needs cleaning", "disgusting", "gross"
    ],
    "location": [
        "far from", "long walk", "difficult to find", "hard to find",
        "unsafe area", "sketchy", "dangerous", "scary neighborhood",
        "not central", "middle of nowhere", "bad location",
        "misleading location", "not as described", "wrong address"
    ],
    "amenities": [
        "broken", "not working", "didn't work", "does not work",
        "missing", "no wifi", "wifi didn't work", "no hot water",
        "cold water", "air conditioning broken", "ac not working",
        "heater broken", "tv broken", "fridge broken",
        "false advertising", "not as pictured", "different from photos"
    ],
    "host": [
        "unresponsive", "no response", "didn't reply", "never answered",
        "rude host", "unfriendly", "unhelpful", "poor communication",
        "late check-in", "waited", "no one there", "couldn't reach",
        "unprofessional", "disorganized"
    ],
    "value": [
        "overpriced", "not worth", "rip off", "ripoff", "too expensive",
        "waste of money", "poor value", "bad value"
    ]
}

# ============================================================
# PRAISE KEYWORDS (Reality Signals - Positive)
# ============================================================

PRAISE_KEYWORDS = {
    "quiet": [
        "quiet", "peaceful", "silent", "tranquil", "serene",
        "slept well", "good sleep", "restful", "no noise"
    ],
    "clean": [
        "spotless", "immaculate", "very clean", "super clean",
        "pristine", "sparkling", "fresh", "well maintained"
    ],
    "location": [
        "perfect location", "great location", "excellent location",
        "central", "walkable", "close to everything", "easy to find",
        "safe area", "nice neighborhood", "convenient"
    ],
    "amenities": [
        "well equipped", "everything you need", "great amenities",
        "comfortable bed", "nice bathroom", "modern", "as described"
    ],
    "host": [
        "helpful host", "friendly host", "responsive", "quick response",
        "welcoming", "accommodating", "went above and beyond",
        "great communication", "easy check-in"
    ],
    "value": [
        "great value", "worth every penny", "good price",
        "reasonable", "bargain", "affordable"
    ]
}

In [0]:
# Create regex patterns for each category
def create_pattern(keywords):
    """Create regex pattern from keyword list"""
    # Escape special regex characters and join with OR
    escaped = [k.replace("'", "\\'") for k in keywords]
    return "(?i)(" + "|".join(escaped) + ")"

# Create complaint patterns
complaint_patterns = {cat: create_pattern(kws) for cat, kws in COMPLAINT_KEYWORDS.items()}
praise_patterns = {cat: create_pattern(kws) for cat, kws in PRAISE_KEYWORDS.items()}

print("Complaint patterns created for:", list(complaint_patterns.keys()))
print("Praise patterns created for:", list(praise_patterns.keys()))

Complaint patterns created for: ['noise', 'cleanliness', 'location', 'amenities', 'host', 'value']
Praise patterns created for: ['quiet', 'clean', 'location', 'amenities', 'host', 'value']


In [0]:
# Start with hotel_enriched and add lowercase reviews
hotel_with_nlp = hotel_enriched.withColumn(
    "reviews_lower",
    lower(col("all_reviews_text"))
)

# Add complaint detection for each category
for category, pattern in complaint_patterns.items():
    hotel_with_nlp = hotel_with_nlp.withColumn(
        f"complaint_{category}",
        when(
            col("reviews_lower").rlike(pattern),
            True
        ).otherwise(False)
    )

# Add praise detection for each category
for category, pattern in praise_patterns.items():
    hotel_with_nlp = hotel_with_nlp.withColumn(
        f"praise_{category}",
        when(
            col("reviews_lower").rlike(pattern),
            True
        ).otherwise(False)
    )

# Check results
print("=== Sample Complaint/Praise Detection ===")
display(
    hotel_with_nlp
    .filter(col("all_reviews_text").isNotNull())
    .select(
        "hotel_id", "city", "title", "review_score",
        "complaint_noise", "complaint_cleanliness", "complaint_location",
        "praise_quiet", "praise_clean", "praise_location"
    )
    .limit(20)
)

=== Sample Complaint/Praise Detection ===


hotel_id,city,title,review_score,complaint_noise,complaint_cleanliness,complaint_location,praise_quiet,praise_clean,praise_location
188199,Rome,Residenza Sciarra B&B,8.0,true,true,false,true,true,true
11318279,Rome,MONTI QUIET AND SPACIOUS APARTMENT,9.1,false,false,false,false,false,false
8591596,Rome,Terrazza Giuliana,9.2,false,true,true,true,true,true
10963876,Rome,Trevi Apartment,8.2,false,false,false,true,false,true
7250866,Rome,Casa Vacanze Campo de' Fiori,8.0,true,true,false,true,false,true
11549337,Rome,Grottino 3a Apartment,8.7,false,false,false,false,true,true
12950226,Rome,Metro Garbatella Casa Caterina,9.6,false,false,false,false,false,false
12086299,Rome,null,null,false,false,false,true,false,true
8914023,Rome,Domus Otto,8.8,false,true,false,false,true,true
2078791,Rome,Rental in Rome Otranto Relax,8.2,false,true,false,true,false,false


In [0]:
from pyspark.sql.functions import expr

# Count actual occurrences (not just boolean)
# This gives us intensity of complaints

def count_matches(text_col, pattern):
    """Count how many times pattern matches in text"""
    return F.size(F.expr(f"regexp_extract_all({text_col}, '{pattern}', 0)"))

# Add complaint counts
for category, keywords in COMPLAINT_KEYWORDS.items():
    # Simple word count approach
    hotel_with_nlp = hotel_with_nlp.withColumn(
        f"complaint_{category}_count",
        F.size(
            F.array_remove(
                F.transform(
                    F.split(col("reviews_lower"), "\\s+"),
                    lambda x: when(
                        x.rlike(create_pattern(keywords)),
                        x
                    )
                ),
                None
            )
        )
    )

# Add praise counts
for category, keywords in PRAISE_KEYWORDS.items():
    hotel_with_nlp = hotel_with_nlp.withColumn(
        f"praise_{category}_count",
        F.size(
            F.array_remove(
                F.transform(
                    F.split(col("reviews_lower"), "\\s+"),
                    lambda x: when(
                        x.rlike(create_pattern(keywords)),
                        x
                    )
                ),
                None
            )
        )
    )

### Calulcate complatins metrics

In [0]:
# Calculate total complaints and praises
hotel_with_nlp = hotel_with_nlp.withColumn(
    "total_complaints",
    col("complaint_noise_count") + col("complaint_cleanliness_count") + 
    col("complaint_location_count") + col("complaint_amenities_count") + 
    col("complaint_host_count") + col("complaint_value_count")
).withColumn(
    "total_praises",
    col("praise_quiet_count") + col("praise_clean_count") + 
    col("praise_location_count") + col("praise_amenities_count") + 
    col("praise_host_count") + col("praise_value_count")
)

# Calculate sentiment ratio (positive / total)
hotel_with_nlp = hotel_with_nlp.withColumn(
    "sentiment_ratio",
    when(
        (col("total_complaints") + col("total_praises")) > 0,
        col("total_praises") / (col("total_complaints") + col("total_praises"))
    ).otherwise(0.5)  # Neutral if no keywords found
)

# Calculate complaint ratio (complaints / total keywords)
hotel_with_nlp = hotel_with_nlp.withColumn(
    "complaint_ratio",
    when(
        (col("total_complaints") + col("total_praises")) > 0,
        col("total_complaints") / (col("total_complaints") + col("total_praises"))
    ).otherwise(0.0)
)

# Check distribution
print("=== Sentiment Distribution ===")
display(
    hotel_with_nlp
    .filter(col("all_reviews_text").isNotNull())
    .select(
        F.round(F.avg("sentiment_ratio"), 2).alias("avg_sentiment"),
        F.round(F.avg("complaint_ratio"), 2).alias("avg_complaint_ratio"),
        F.round(F.avg("total_complaints"), 1).alias("avg_complaints"),
        F.round(F.avg("total_praises"), 1).alias("avg_praises")
    )
)

=== Sentiment Distribution ===


avg_sentiment,avg_complaint_ratio,avg_complaints,avg_praises
0.5,0.0,-6.0,-6.0


In [0]:
# Complaint analysis by city
print("=== Complaints by City ===")
display(
    hotel_with_nlp
    .filter(col("all_reviews_text").isNotNull())
    .groupBy("city")
    .agg(
        spark_count("*").alias("hotels_with_reviews"),
        F.round(spark_avg("complaint_ratio"), 3).alias("avg_complaint_ratio"),
        F.round(spark_avg("sentiment_ratio"), 3).alias("avg_sentiment_ratio"),
        F.round(spark_avg(col("complaint_noise").cast("int")), 3).alias("noise_complaint_rate"),
        F.round(spark_avg(col("complaint_cleanliness").cast("int")), 3).alias("clean_complaint_rate"),
        F.round(spark_avg(col("complaint_location").cast("int")), 3).alias("location_complaint_rate"),
        F.round(spark_avg(col("complaint_amenities").cast("int")), 3).alias("amenity_complaint_rate"),
        F.round(spark_avg(col("complaint_host").cast("int")), 3).alias("host_complaint_rate")
    )
    .orderBy("city")
)

=== Complaints by City ===


city,hotels_with_reviews,avg_complaint_ratio,avg_sentiment_ratio,noise_complaint_rate,clean_complaint_rate,location_complaint_rate,amenity_complaint_rate,host_complaint_rate
Amsterdam,985,0.0,0.5,0.361,0.53,0.169,0.105,0.015
Bangkok,2953,0.0,0.5,0.265,0.355,0.147,0.066,0.014
Dubai,15250,0.0,0.5,0.032,0.077,0.01,0.018,0.004
Eilat,680,0.0,0.5,0.09,0.15,0.047,0.062,0.01
Haifa,262,0.0,0.5,0.13,0.221,0.05,0.065,0.019
London,15556,0.0,0.5,0.162,0.264,0.047,0.074,0.009
New York,2255,0.0,0.5,0.153,0.197,0.036,0.048,0.009
Rome,16883,0.0,0.5,0.247,0.474,0.105,0.09,0.027
Tel Aviv,955,0.0,0.5,0.118,0.173,0.022,0.044,0.007
Tokyo,6864,0.0,0.5,0.128,0.291,0.091,0.033,0.008


In [0]:
# Create gap signals by comparing claims vs complaints

hotel_with_gaps = hotel_with_nlp.withColumn(
    # NOISE GAP: Claims quiet but has noise complaints
    "noise_gap_signal",
    when(
        (col("claims_quiet") == True) & (col("complaint_noise") == True),
        1.0  # High gap
    ).when(
        (col("claims_quiet") == True) & (col("praise_quiet") == True),
        -0.5  # Negative gap (better than expected)
    ).otherwise(0.0)
).withColumn(
    # CLEANLINESS GAP: High review score but cleanliness complaints
    "cleanliness_gap_signal",
    when(
        (col("review_score") >= 8) & (col("complaint_cleanliness") == True),
        1.0
    ).when(
        (col("review_score") >= 8) & (col("praise_clean") == True),
        -0.5
    ).otherwise(0.0)
).withColumn(
    # LOCATION GAP: Claims central but location complaints
    "location_gap_signal",
    when(
        (col("claims_central") == True) & (col("complaint_location") == True),
        1.0
    ).when(
        (col("claims_central") == True) & (col("praise_location") == True),
        -0.5
    ).otherwise(0.0)
).withColumn(
    # AMENITY GAP: Many claimed amenities but complaints
    "amenity_gap_signal",
    when(
        (col("amenities_count") >= 5) & (col("complaint_amenities") == True),
        1.0
    ).when(
        (col("amenities_count") >= 5) & (col("praise_amenities") == True),
        -0.5
    ).otherwise(0.0)
).withColumn(
    # HOST GAP: Good manager score but host complaints
    "host_gap_signal",
    when(
        (col("manager_score") >= 8) & (col("complaint_host") == True),
        1.0
    ).when(
        (col("manager_score") >= 8) & (col("praise_host") == True),
        -0.5
    ).otherwise(0.0)
)

# Check gap signals
print("=== Gap Signals by City ===")
display(
    hotel_with_gaps
    .filter(col("all_reviews_text").isNotNull())
    .groupBy("city")
    .agg(
        spark_count("*").alias("hotels"),
        F.round(spark_avg("noise_gap_signal"), 3).alias("avg_noise_gap"),
        F.round(spark_avg("cleanliness_gap_signal"), 3).alias("avg_clean_gap"),
        F.round(spark_avg("location_gap_signal"), 3).alias("avg_location_gap"),
        F.round(spark_avg("amenity_gap_signal"), 3).alias("avg_amenity_gap"),
        F.round(spark_avg("host_gap_signal"), 3).alias("avg_host_gap")
    )
    .orderBy("city")
)

=== Gap Signals by City ===


city,hotels,avg_noise_gap,avg_clean_gap,avg_location_gap,avg_amenity_gap,avg_host_gap
Amsterdam,985,0.017,0.168,-0.14,-0.101,-0.111
Bangkok,2953,0.006,0.139,-0.018,-0.05,-0.021
Dubai,15250,0.001,0.007,-0.033,-0.014,-0.018
Eilat,680,0.003,0.015,-0.018,-0.009,-0.015
Haifa,262,0.029,0.027,-0.052,-0.021,-0.027
London,15556,0.011,0.092,-0.093,-0.029,-0.063
New York,2255,0.003,0.039,-0.078,-0.014,-0.017
Rome,16883,0.008,0.199,-0.103,-0.077,-0.056
Tel Aviv,955,0.006,0.01,-0.077,-0.023,-0.032
Tokyo,6864,0.002,0.118,-0.034,-0.049,-0.018


### Cache and save

In [0]:
# Cache the result
hotel_with_gaps.cache()

print(f"Total hotels processed: {hotel_with_gaps.count():,}")

# Show final schema (new NLP columns)
nlp_columns = [c for c in hotel_with_gaps.columns if 'complaint' in c or 'praise' in c or 'gap_signal' in c or 'sentiment' in c]
print(f"\nNLP columns added: {len(nlp_columns)}")
for c in nlp_columns:
    print(f"  - {c}")

Total hotels processed: 62,643

NLP columns added: 33
  - complaint_noise
  - complaint_cleanliness
  - complaint_location
  - complaint_amenities
  - complaint_host
  - complaint_value
  - praise_quiet
  - praise_clean
  - praise_location
  - praise_amenities
  - praise_host
  - praise_value
  - complaint_noise_count
  - complaint_cleanliness_count
  - complaint_location_count
  - complaint_amenities_count
  - complaint_host_count
  - complaint_value_count
  - praise_quiet_count
  - praise_clean_count
  - praise_location_count
  - praise_amenities_count
  - praise_host_count
  - praise_value_count
  - total_complaints
  - total_praises
  - sentiment_ratio
  - complaint_ratio
  - noise_gap_signal
  - cleanliness_gap_signal
  - location_gap_signal
  - amenity_gap_signal
  - host_gap_signal


In [0]:
# Save to Gold layer
hotel_with_gaps.write \
    .mode("overwrite") \
    .partitionBy("city") \
    .parquet("dbfs:/FileStore/project/gold/hotel_with_nlp")

print("Saved to: dbfs:/FileStore/project/gold/hotel_with_nlp")

Saved to: dbfs:/FileStore/project/gold/hotel_with_nlp


### Sanity check on high gap hotels

In [0]:
# Find hotels with highest gap signals
print("=== Hotels with Highest Noise Gap ===")
display(
    hotel_with_gaps
    .filter(
        (col("claims_quiet") == True) & 
        (col("complaint_noise") == True)
    )
    .select(
        "hotel_id", "city", "title", "review_score",
        "claims_quiet", "complaint_noise", "nightlife_count_500m",
        "noise_gap_signal", "all_reviews_text"
    )
    .orderBy(F.desc("nightlife_count_500m"))
    .limit(10)
)

=== Hotels with Highest Noise Gap ===


hotel_id city title review_score claims_quiet complaint_noise nightlife_count_500m noise_gap_signal all_reviews_text 10471441 Tokyo Ueno Retro Flower - Ueno Okachimachi 8.6 true true 590 1.0 It has very spacious Japanese type rooms and all the amenities that you need for simple cooking and also for laundry; it is also within short walking distance to the train and subway stations The property is actually very close to night clubs, thus it is not suitable for family with young children to walk pass hostesses peddling for customers; the noises for drunk customers of the night clubs late at night were a nuisance ||| The house is very great,and near by Ueno .
If we will visit to japan,we will stay again. ||| Great washing and drying machine. 

All as expected It smell a bit to food, maybe from a restaurant underneath? ||| 넓은 공간
보통 오사카 후쿠오카쪽 호텔들은 너무 좁아서 침대위에서 캐리어를 풀어야했는데 충분한 공간이라 여유가 있었음
방도 여유가 있음

접근성
위치가 우에노공원과 가깝고 주변에 슈퍼마켓들이 있어서 편리함
돈키호테도 5분거리
히키하바라도 걸어 갈만함

편리함
요리를 할수있는 도구들이 비치되어 있고 뜨거운 물이 잘 나옴
전기코드가 여기저기있어서 충전등이 편리함

다다미방
일본 다다미방을 체험할수있고 컨디션도 좋음

빠른응대
질문등을 하면 호스트의 응대가 빠른고 친절함 소음
시내복판이라 가끔 싸이렌차가 지나다니고 새벽에 술먹고 주정부리는 사람들의 소리가 들림
그래서 귀마개가 준비되어 있음
예민한 사람은 거슬릴수있지만 무시할정도

분위기
밤이되면 길거리주변에 호객행위하는 사람들이 길거리 가장자리에 많음
일본시내라면 흔한광경 ||| 교통편리,편안한 숙소와 편의시설, 주변에 마트도 가깝고 여러모로 좋았습니다 다시와도 이곳에 머물고 싶네요 감사합니다 ^^* ||| ที่พักเดินทางสะดวก ไกล้แหล่งช๊อปปิ้ง ข้ามถนนก็เป็นตลาดอุเอโนะ 200Mเป็นดองกี้ หาของกินง่าย ไกล้รถไฟทั้งใต้ดินและJR ขากลับลากกระเป๋าขึ้นKEISEI ประมาณ 450Mได้อยู่ จาน เช้อน เครื่องครัว ตู้เย็น ไมโครเวฟ ค.ซักผ้า ผ้าเช็ดตัวผื่นใหญ่-เล็ก ครบจำนวนคน มีระเบียงให้ตากผ้าด้วย มี2ห้องนอนและห้องเล็กๆที่มีโซฟาอีก1ห้อง นอนบนเสื่อ ฟูก-ผ้าห่มนุ่มอุ่นนอนสบาย เป็นอพาร์ทเม้นท์ชั้นละ1ห้อง มีลิฟท์ขึ้นไปชั้น 6 เป็นส่วนตัวดี เช็คอิน-เช็คเอ้าท์ด้วยตัวเองง่าย ไม่ผิดหวังเลยคะ ห้องน้ำกับห้องส้วมแยกกันตามสไตล์ห้องน้ำญี่ปุ่น ถ้านอน8คนก็ต้องค่อยๆเรียงกันใช้ 245089 Tokyo null null true true 248 1.0 The staff was super nice and friendly. They managed to check me in hours before my check in time which is such a surprise for me. The location was quite convenient, close to the Ueno station and has lots of restaurants around. The morning cleaning can be a bit loud because it’s an old building. If you are a morning person, there won’t be any problem. But if you are used to getting up late, make sure you have your earplug on. ||| Location is great, short walk to Ueno train station and close to some restaurants, was able to leave luggage to collect later, after check out. Stayed on two occasions, first room was very small but the next one was a bit better. No lift, have to carry luggage up stairs. ||| Near with train statetion and comfortable
Good service and friendly staff ||| We love this place. It's clean, neat, cozy, quiet. To be in Tokyo, room had good space. The beds were so, so, so comfortable and perfect to rest after walking the whole day. It's near to Ueno station, and a great restaurants area. They have 24-hours reception. Just keep in mind the building doesn't have elevator. ||| great location, nice staff, room is the best in this area, clean. i surely will be back again next time. no elevator ||| Really nice neighborhood, super close to the Ueno station The rooms are super small, the double bed is more like 1.5 ||| I loved how close this hotel was to Ueno station. Only a 6 min walk. They also let us keep our luggage in the dining area when we had a few hours before our bullet train left and when we had to checkout. Also, the one staff member made us hot water for our ramen without us even asking. So sweet! They also cleaned our room every day. There isn’t anything to complain about. ||| Location is near to train stations, convenience stores and restaurants. Room is quite small ||| Excellent Location. Huge bedroom size. Excellent staffs. Bathroom is rather small and has a bad smell from the drainage. There are some stains on room carpet. ||| The location of the hotel is near to many train stations. The accessories needed inside the hotel is sufficient enough. The s

In [0]:
# Hotels claiming central but with location complaints
print("=== Hotels with Location Gap ===")
display(
    hotel_with_gaps
    .filter(
        (col("claims_central") == True) & 
        (col("complaint_location") == True)
    )
    .select(
        "hotel_id", "city", "title", "review_score",
        "claims_central", "complaint_location", 
        "nearest_transport_m", "location_gap_signal"
    )
    .orderBy(F.desc("nearest_transport_m"))
    .limit(10)
)

=== Hotels with Location Gap ===


hotel_id,city,title,review_score,claims_central,complaint_location,nearest_transport_m,location_gap_signal
11992116,Bangkok,null,null,true,true,983.5864131174123,1.0
1104162,Bangkok,Marigold Sukhumvit,7.8,true,true,961.1039442698583,1.0
1424491,Bangkok,Yotaka Boutique Hotel Bangkok,6.9,true,true,954.6729768293508,1.0
2174967,Bangkok,Roong-Arun Apartment,8.6,true,true,934.5872368997292,1.0
12070055,Tokyo,Bijou Suites Cocoa,9.5,true,true,916.6167803764079,1.0
27514,Bangkok,Miracle Grand Convention Hotel,8.2,true,true,877.0713079803478,1.0
2021360,Bangkok,Leenova Hotel,7.1,true,true,869.740463908671,1.0
4255728,Bangkok,Casa Luxe Hotel and Resident,8.3,true,true,868.571589607462,1.0
11518128,Bangkok,"206HiddenNest, Local Room with Street Food only 15 min walk to Skytrain",8.5,true,true,861.9938310106081,1.0
7926230,Bangkok,37 Nature House,8.6,true,true,860.2180104328675,1.0


## Part 2

here we do basic NLP - pos vs neg

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, lower, length, size, split, lit
from pyspark.sql.functions import sum as spark_sum, avg as spark_avg, count as spark_count

# ============================================================
# SENTIMENT WORD LISTS (Apostrophes removed/replaced)
# ============================================================

# Positive words (general sentiment)
POSITIVE_WORDS = [
    # Strong positive
    "amazing", "excellent", "fantastic", "wonderful", "incredible", "outstanding",
    "perfect", "exceptional", "superb", "brilliant", "magnificent", "marvelous",
    
    # Medium positive
    "great", "lovely", "beautiful", "comfortable", "pleasant", "nice", "good",
    "clean", "spacious", "modern", "cozy", "friendly", "helpful", "recommended",
    "enjoyed", "loved", "best", "awesome", "delightful", "impressive",
    
    # Light positive
    "fine", "okay", "decent", "adequate", "satisfactory", "convenient",
    "happy", "pleased", "satisfied", "thankful", "grateful"
]

# Negative words (general sentiment) - NO APOSTROPHES
NEGATIVE_WORDS = [
    # Strong negative
    "terrible", "horrible", "awful", "disgusting", "dreadful", "worst",
    "unacceptable", "appalling", "atrocious", "abysmal", "nightmare",
    
    # Medium negative
    "bad", "poor", "dirty", "noisy", "uncomfortable", "disappointing",
    "unpleasant", "rude", "unfriendly", "unhelpful",
    "broken", "damaged", "stained", "smelly", "moldy", "dusty",
    "overpriced", "expensive", "ripoff", "scam", "waste",
    
    # Light negative
    "mediocre", "average", "underwhelming", "lacking", "basic",
    "small", "cramped", "old", "dated", "tired", "worn",
    
    # Complaint phrases (no apostrophes)
    "avoid", "regret", "mistake", "ruined", "frustrated", "angry", "upset"
]

# Negators - simplified without contractions
NEGATORS = [
    "not", "no", "never", "neither", "nobody", "nothing", "nowhere",
    "hardly", "barely", "without"
]

print(f"Positive words: {len(POSITIVE_WORDS)}")
print(f"Negative words: {len(NEGATIVE_WORDS)}")
print(f"Negators: {len(NEGATORS)}")

Positive words: 43
Negative words: 50
Negators: 10


In [0]:
# Load the data
hotel_with_nlp = spark.read.parquet("dbfs:/FileStore/project/gold/hotel_with_nlp")

print(f"Total hotels: {hotel_with_nlp.count():,}")

# Create simple regex patterns (no special characters)
positive_pattern = "(" + "|".join(POSITIVE_WORDS) + ")"
negative_pattern = "(" + "|".join(NEGATIVE_WORDS) + ")"
negator_pattern = "(" + "|".join(NEGATORS) + ")"

# Add lowercase reviews
hotel_with_sentiment = hotel_with_nlp.withColumn(
    "reviews_lower",
    lower(col("all_reviews_text"))
)

# Count positive words using regexp_extract_all
hotel_with_sentiment = hotel_with_sentiment.withColumn(
    "positive_word_count",
    when(
        col("reviews_lower").isNotNull(),
        F.size(F.expr(f"regexp_extract_all(reviews_lower, '(?i)\\\\b({positive_pattern})\\\\b', 0)"))
    ).otherwise(0)
)

# Count negative words
hotel_with_sentiment = hotel_with_sentiment.withColumn(
    "negative_word_count",
    when(
        col("reviews_lower").isNotNull(),
        F.size(F.expr(f"regexp_extract_all(reviews_lower, '(?i)\\\\b({negative_pattern})\\\\b', 0)"))
    ).otherwise(0)
)

# Count negators
hotel_with_sentiment = hotel_with_sentiment.withColumn(
    "negator_count",
    when(
        col("reviews_lower").isNotNull(),
        F.size(F.expr(f"regexp_extract_all(reviews_lower, '(?i)\\\\b({negator_pattern})\\\\b', 0)"))
    ).otherwise(0)
)

# Check counts
print("=== Sentiment Word Counts Sample ===")
display(
    hotel_with_sentiment
    .filter(col("all_reviews_text").isNotNull())
    .select(
        "hotel_id", "city", "title", "review_score",
        "positive_word_count", "negative_word_count", "negator_count"
    )
    .limit(20)
)

Total hotels: 62,643
=== Sentiment Word Counts Sample ===


hotel_id,city,title,review_score,positive_word_count,negative_word_count,negator_count
188199,Rome,Residenza Sciarra B&B,8.0,23,4,6
11318279,Rome,MONTI QUIET AND SPACIOUS APARTMENT,9.1,6,0,1
8591596,Rome,Terrazza Giuliana,9.2,47,1,7
10963876,Rome,Trevi Apartment,8.2,0,0,0
7250866,Rome,Casa Vacanze Campo de' Fiori,8.0,28,5,8
11549337,Rome,Grottino 3a Apartment,8.7,18,2,6
12950226,Rome,Metro Garbatella Casa Caterina,9.6,0,0,0
12086299,Rome,null,null,6,0,2
8914023,Rome,Domus Otto,8.8,8,0,0
2078791,Rome,Rental in Rome Otranto Relax,8.2,14,2,3


In [0]:
# ============================================================
# CELL: NORMALIZE WORD COUNTS (Fix Reviewer Bias)
# ============================================================

# Calculate total words and density to avoid bias from long reviews
hotel_with_sentiment = hotel_with_sentiment.withColumn(
    "review_word_count", 
    F.size(F.split(col("all_reviews_text"), "\\s+"))
).withColumn(
    "positive_density", 
    when(col("review_word_count") > 0, col("positive_word_count") / col("review_word_count")).otherwise(0)
).withColumn(
    "negative_density", 
    when(col("review_word_count") > 0, col("negative_word_count") / col("review_word_count")).otherwise(0)
)

print("Added normalized density features to remove review length bias.")

Added normalized density features to remove review length bias.


### Calculate sentimnet score

In [0]:
# Calculate sentiment scores

hotel_with_sentiment = hotel_with_sentiment.withColumn(
    # Total sentiment words
    "total_sentiment_words",
    col("positive_word_count") + col("negative_word_count")
).withColumn(
    # Basic sentiment ratio: positive / total (0 to 1)
    # 1 = all positive, 0 = all negative, 0.5 = neutral
    "sentiment_score",
    when(
        col("total_sentiment_words") > 0,
        col("positive_word_count") / col("total_sentiment_words")
    ).otherwise(0.5)  # Neutral if no sentiment words
).withColumn(
    # Adjusted sentiment: account for negators
    # Negators often flip sentiment, so more negators = less reliable positive sentiment
    "negator_ratio",
    when(
        col("total_sentiment_words") > 0,
        col("negator_count") / col("total_sentiment_words")
    ).otherwise(0.0)
).withColumn(
    # Sentiment polarity: ranges from -1 (very negative) to +1 (very positive)
    "sentiment_polarity",
    when(
        col("total_sentiment_words") > 0,
        (col("positive_word_count") - col("negative_word_count")) / col("total_sentiment_words")
    ).otherwise(0.0)
).withColumn(
    # Sentiment intensity: how "emotional" are the reviews?
    # More sentiment words per review = higher intensity
    "sentiment_intensity",
    when(
        col("num_reviews_available") > 0,
        col("total_sentiment_words") / col("num_reviews_available")
    ).otherwise(0.0)
).withColumn(
    # Adjusted score: penalize if many negators present
    # Logic: "not good" should reduce sentiment even though "good" is positive
    "sentiment_score_adjusted",
    when(
        col("total_sentiment_words") > 0,
        col("sentiment_score") * (1 - 0.5 * col("negator_ratio"))
    ).otherwise(0.5)
)

print("Sentiment scores calculated!")

Sentiment scores calculated!


In [0]:
# Check if our sentiment correlates with actual review scores
print("=== Sentiment vs Review Score Correlation ===")
display(
    hotel_with_sentiment
    .filter(col("all_reviews_text").isNotNull())
    .select(
        F.round(F.corr("sentiment_score", "review_score"), 3).alias("sentiment_vs_review_corr"),
        F.round(F.corr("sentiment_polarity", "review_score"), 3).alias("polarity_vs_review_corr"),
        F.round(F.corr("negative_word_count", "review_score"), 3).alias("negative_vs_review_corr")
    )
)

=== Sentiment vs Review Score Correlation ===


sentiment_vs_review_corr,polarity_vs_review_corr,negative_vs_review_corr
0.453,0.453,0.15


In [0]:
# Sentiment distribution by review score bucket
print("=== Sentiment by Review Score Bucket ===")
display(
    hotel_with_sentiment
    .filter(col("all_reviews_text").isNotNull())
    .withColumn(
        "score_bucket",
        when(col("review_score") >= 9, "9-10 (Excellent)")
        .when(col("review_score") >= 8, "8-9 (Very Good)")
        .when(col("review_score") >= 7, "7-8 (Good)")
        .when(col("review_score") >= 6, "6-7 (Average)")
        .otherwise("< 6 (Poor)")
    )
    .groupBy("score_bucket")
    .agg(
        spark_count("*").alias("hotels"),
        F.round(spark_avg("sentiment_score"), 3).alias("avg_sentiment"),
        F.round(spark_avg("sentiment_polarity"), 3).alias("avg_polarity"),
        F.round(spark_avg("positive_word_count"), 1).alias("avg_positive_words"),
        F.round(spark_avg("negative_word_count"), 1).alias("avg_negative_words")
    )
    .orderBy(F.desc("score_bucket"))
)

=== Sentiment by Review Score Bucket ===


score_bucket,hotels,avg_sentiment,avg_polarity,avg_positive_words,avg_negative_words
< 6 (Poor),42314,0.657,0.315,5.5,0.5
9-10 (Excellent),6706,0.858,0.717,15.0,0.8
8-9 (Very Good),7779,0.876,0.752,18.5,1.7
7-8 (Good),4221,0.837,0.675,14.4,1.6
6-7 (Average),1623,0.794,0.589,9.3,1.1


### View By city

In [0]:
# Sentiment by city
print("=== Sentiment by City ===")
display(
    hotel_with_sentiment
    .filter(col("all_reviews_text").isNotNull())
    .groupBy("city")
    .agg(
        spark_count("*").alias("hotels"),
        F.round(spark_avg("sentiment_score"), 3).alias("avg_sentiment"),
        F.round(spark_avg("sentiment_polarity"), 3).alias("avg_polarity"),
        F.round(spark_avg("sentiment_intensity"), 2).alias("avg_intensity"),
        F.round(spark_avg("positive_word_count"), 1).alias("avg_positive"),
        F.round(spark_avg("negative_word_count"), 1).alias("avg_negative")
    )
    .orderBy(F.desc("avg_sentiment"))
)

=== Sentiment by City ===


city,hotels,avg_sentiment,avg_polarity,avg_intensity,avg_positive,avg_negative
Amsterdam,985,0.869,0.738,2.56,22.6,1.9
Rome,16883,0.815,0.629,1.68,13.9,1.1
Bangkok,2953,0.776,0.551,1.76,13.9,1.1
London,15556,0.731,0.462,1.44,8.7,0.8
Haifa,262,0.721,0.442,1.11,7.8,0.6
Eilat,680,0.706,0.412,0.73,5.9,0.4
Tokyo,6864,0.695,0.39,1.19,7.4,1.0
Tel Aviv,955,0.657,0.314,0.96,6.0,0.5
New York,2255,0.643,0.286,0.99,7.6,0.8
Dubai,15250,0.618,0.236,0.65,2.7,0.1


### Create indactors based on sentiment

In [0]:
# Create sentiment gap: expected sentiment vs actual
# High review_score should have high sentiment, if not = gap

hotel_with_sentiment = hotel_with_sentiment.withColumn(
    # Expected sentiment based on review score
    "expected_sentiment",
    (col("review_score") - 5) / 5  # Maps 5->0, 10->1, linear
).withColumn(
    # Sentiment gap: positive = reviews worse than score suggests
    "sentiment_gap",
    col("expected_sentiment") - col("sentiment_polarity")
).withColumn(
    # Sentiment mismatch flag: high score but negative sentiment
    "sentiment_mismatch",
    when(
        (col("review_score") >= 8) & (col("sentiment_polarity") < 0),
        True
    ).when(
        (col("review_score") < 6) & (col("sentiment_polarity") > 0.3),
        True  # Also flag: low score but very positive sentiment (unusual)
    ).otherwise(False)
)

# Find hotels with sentiment mismatch
print("=== Hotels with Sentiment Mismatch (High Score, Negative Sentiment) ===")
display(
    hotel_with_sentiment
    .filter(
        (col("review_score") >= 8) & 
        (col("sentiment_polarity") < 0) &
        (col("all_reviews_text").isNotNull())
    )
    .select(
        "hotel_id", "city", "title", 
        "review_score", "sentiment_score", "sentiment_polarity",
        "positive_word_count", "negative_word_count"
    )
    .orderBy("sentiment_polarity")
    .limit(15)
)

=== Hotels with Sentiment Mismatch (High Score, Negative Sentiment) ===


hotel_id,city,title,review_score,sentiment_score,sentiment_polarity,positive_word_count,negative_word_count
8402753,Tel Aviv,DeBlox living - Ibn Gabirol Apartments,8.1,0.0,-1.0,0,1
11394379,Rome,Casa dei fiori by GHOR,8.7,0.0,-1.0,0,1
13158263,London,Modern Flat with Balcony next to Station by CTO Serviced Accommodation,10.0,0.0,-1.0,0,1
11006272,Rome,Moderno bilocale con terrazzo,9.1,0.0,-1.0,0,1
13056399,Tokyo,innnn higashinagasaki,9.0,0.0,-1.0,0,1
9195537,Rome,Casa e Appartamenti per Vacanze Jenner,9.1,0.0,-1.0,0,1
12466787,London,Cozy City Apartment in London,8.0,0.0,-1.0,0,1
5850839,Rome,Charming Apartment with Panoramic View in the Heart of Rome,8.0,0.0,-1.0,0,1
13031471,Tokyo,incense上野根岸201,8.0,0.0,-1.0,0,2
11080848,Rome,iFlat The Tiber Gem,8.5,0.0,-1.0,0,1


In [0]:
# Final sentiment summary
print("=== Overall Sentiment Summary ===")
display(
    hotel_with_sentiment
    .filter(col("all_reviews_text").isNotNull())
    .select(
        F.round(spark_avg("sentiment_score"), 3).alias("avg_sentiment_score"),
        F.round(spark_avg("sentiment_polarity"), 3).alias("avg_polarity"),
        F.round(spark_avg("sentiment_intensity"), 2).alias("avg_intensity"),
        F.round(spark_avg("sentiment_gap"), 3).alias("avg_sentiment_gap"),
        spark_sum(col("sentiment_mismatch").cast("int")).alias("total_mismatches"),
        spark_count("*").alias("total_hotels")
    )
)

=== Overall Sentiment Summary ===


avg_sentiment_score,avg_polarity,avg_intensity,avg_sentiment_gap,total_mismatches,total_hotels
0.722,0.444,1.29,-0.104,607,62643


In [0]:
# Print new columns added
sentiment_columns = [
    "positive_word_count", "negative_word_count", "intensifier_count", "negator_count",
    "total_sentiment_words", "sentiment_score", "sentiment_polarity", 
    "sentiment_intensity", "sentiment_score_adjusted", "negator_ratio",
    "expected_sentiment", "sentiment_gap", "sentiment_mismatch"
]

print(f"\n=== New Sentiment Columns Added ({len(sentiment_columns)}) ===")
for c in sentiment_columns:
    print(f"  • {c}")


=== New Sentiment Columns Added (13) ===
  • positive_word_count
  • negative_word_count
  • intensifier_count
  • negator_count
  • total_sentiment_words
  • sentiment_score
  • sentiment_polarity
  • sentiment_intensity
  • sentiment_score_adjusted
  • negator_ratio
  • expected_sentiment
  • sentiment_gap
  • sentiment_mismatch


In [0]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.clustering import LDA
from pyspark.ml import Pipeline
from pyspark.sql.functions import regexp_replace, lower, col, udf, trim
from pyspark.sql.types import IntegerType

# ============================================================
# 1. CLEANING & PREPARATION
# ============================================================

# A. Advanced Cleaning: 
# 1. Keep only letters 
# 2. Remove words with 1 or 2 letters (removes 'y', 'n', 'de', 'le', 'it', 'is', 'to')
cleaned_df = hotel_with_sentiment.withColumn(
    "reviews_clean",
    regexp_replace(lower(col("all_reviews_text")), r"[^a-z\s]", " ") 
).withColumn(
    "reviews_clean",
    regexp_replace(col("reviews_clean"), r"\b[a-z]{1,2}\b", " ") # Remove 1-2 letter words
).withColumn(
    "reviews_clean",
    regexp_replace(col("reviews_clean"), r"\s+", " ") 
)

# B. Tokenize
tokenizer = Tokenizer(inputCol="reviews_clean", outputCol="words_raw")

# C. Standard English Stop Words
remover = StopWordsRemover(inputCol="words_raw", outputCol="words_filtered")

# D. Custom Stop Words (Aggressive Cleaning)
# We remove generic positive words so the model finds FEATURES (like 'noise', 'bed') instead.
custom_stopwords = [
    # Generic Hotel Words (Too common to be useful)
    "room", "hotel", "stay", "location", "place", "staff", "everything", 
    "would", "could", "one", "get", "us", "really", "also", "time", "bit",
    "london", "tokyo", "rome", "bangkok", "rooms", "apartment", "host",
    
    # Generic Sentiment (We want topics about FEATURES, not just 'good')
    "great", "good", "nice", "clean", "comfortable", "excellent", "perfect", 
    "amazing", "lovely", "well", "best", "wonderful", "liked", "like", "love",
    "bad", "small", "nothing", "much", "even", "definitely", "highly",
    
    # Common Foreign Stopwords (German, Spanish, French, Italian)
    "und", "die", "der", "das", "ist", "nicht", "sehr", "mit", # German
    "muy", "que", "para", "con", "las", "los", "del", "una",   # Spanish
    "les", "des", "pour", "est", "une", "avec", "très",         # French
    "non", "che", "per", "una", "molto", "sono"                 # Italian
]

remover_custom = StopWordsRemover(inputCol="words_filtered", outputCol="words_clean", stopWords=custom_stopwords)

# E. Vectorize (Top 1000 words, must appear in at least 10 documents)
cv = CountVectorizer(inputCol="words_clean", outputCol="features", vocabSize=1000, minDF=10.0)

# F. LDA Model (Increased maxIter for better convergence)
lda = LDA(k=5, maxIter=20, featuresCol="features")

# PIPELINE EXECUTION


lda_pipeline = Pipeline(stages=[tokenizer, remover, remover_custom, cv, lda])

print("Running LDA Topic Modeling (Aggressive Cleaning)...")
lda_model = lda_pipeline.fit(cleaned_df)
hotel_with_topics = lda_model.transform(cleaned_df)


# VIEW RESULTS


vocab = lda_model.stages[3].vocabulary
topics = lda_model.stages[4].describeTopics(5)
topics_rdd = topics.rdd

print("\n=== Discovered Topics (Refined) ===")
for row in topics_rdd.collect():
    topic_id = row['topic']
    term_indices = row['termIndices']
    words = [vocab[idx] for idx in term_indices]
    print(f"Topic {topic_id}: {words}")

# Assign Dominant Topic
def get_dominant_topic(distribution):
    if distribution is None:
        return -1
    return int(distribution.argmax())

dominant_topic_udf = udf(get_dominant_topic, IntegerType())

hotel_with_topics = hotel_with_topics.withColumn(
    "dominant_topic", 
    dominant_topic_udf(col("topicDistribution"))
)

# Clean up
hotel_with_sentiment = hotel_with_topics.drop("words_raw", "words_filtered", "words_clean", "features", "topicDistribution", "reviews_clean")

print("\nAdded 'dominant_topic' column to dataset.")
display(hotel_with_sentiment.select("hotel_id", "city", "dominant_topic", "all_reviews_text").limit(5))

Running LDA Topic Modeling (Aggressive Cleaning)...
🏃 View run learned-hawk-782 at: https://adb-983293358114278.18.azuredatabricks.net/ml/experiments/200762682747142/runs/236f5c97f28446879f9298ace1383c08
🧪 View experiment at: https://adb-983293358114278.18.azuredatabricks.net/ml/experiments/200762682747142

=== Discovered Topics (Refined) ===
Topic 0: ['metro', 'appartamento', 'posizione', 'bien', 'todo']
Topic 1: ['metro', 'close', 'station', 'helpful', 'restaurants']
Topic 2: ['breakfast', 'friendly', 'helpful', 'close', 'station']
Topic 3: ['close', 'kitchen', 'property', 'easy', 'walk']
Topic 4: ['', 'dubai', 'het', 'view', 'mall']

Added 'dominant_topic' column to dataset.


hotel_id,city,dominant_topic,all_reviews_text
188199,Rome,2,"The location was the absolute best! The pastries in the morning and fresh juices was great. The host communicated through WhatsApp. She responds immediately. The absolute second favorite thing besides the location was the air conditioning! It actually provides cold air. Very, very comfortable. The property itself is a little bit older. So that means smaller elevator, some stairs, and smaller shower. None of these things can truly be changed by the host. The photos are a true representation of the property. I loved it. ||| This is a perfect location with soundproof windows. The room was clean, the bed was comfortable, the coffee was exellent. The breakfast was poor for people who do not eat sugar. The door to the shower room did not close tightly, so there was water on the floor after taking a shower. There were no slippers and cold floor. ||| Quaint neighborhood, easy to get around Trevi. Restaurant downstairs infront of door. Unit on 3rd floor but has an elevator, a little bit of stairs involved but not bad. Breakfast was tasty with expresso. We used Uber to get around, It was difficult to get taxis. Host was great! Old building retro fitted with small elevator, but it is Rome and you have to be flexible. the rest was wonderful ||| Cute B&B. Near Trevi fountain. Great restaurant right outside the door. Unit was clean. Manager set up airport driver for us and the service was excellent. Elevators stopped mid floor. Had to carry luggage up a flight on arrival and down a flight on departure. ||| Staff were always available to help through whatsapp. The place is very centrally located. The place is very safe. Shower was narrow and a/c was weak. ||| Centrally located to walk to all of the major sights. Restaurants right outside the door. Was great for late night snacks and drinks. Very safe. Nothing ||| Very clean, beautiful room. The windows kept out all the street noise. ||| Desirable location. Instructions for entrance very detailed and easy to understand. Very clean. ||| The beds were the most cozy and comfortable. The room was quiet and we got some much needed rest after a fun day and night in Rome. The staff was so helpful in helping us get a taxi to the port after our stay. Excellent customer service. The road to get a taxi is busy, no place for taxi to stop. Otherwise perfect place. ||| Beautiful place and location. Best directions and explanations for the place complete with videos. A little pricey but with it."
11318279,Rome,0,"Location is perfect, walking distance to Colloseum, Termini, and other good places. The apartment is clean, spacious with good ammenities, and has an elevator! The host was also very supportive (thanks Olivia!) Nothing major, but If only there's salt and pepper in the table that will be very helpful to us. ||| Appartement très confortable et très bien situé très propre. Je recommande cette location . ||| Llegamos a lo de Olivia luego de una muy mala experiencia con otro apartamento y nos sentimos en el paraíso. El barrio es cerca de todo . Es espacioso, silencioso, bien decorado, cómodo y bien equipado. Los baños son hermosos y con buenas toallas y shampoo! Tiene lavadora que nos resulta muy necesario. Buena vajilla, tiene tres televisores y espacio de guardado Es súper recomendable ! Nada para criticar. ||| Pentru noi locația a fost perfecta, aproape de gara Termini și la o distanta foarte mica de Colloseum. Ne am putut deplasa pe jos, fiind nevoie sa luam metroul doar pentru Vatican. Deși nu aveam foarte mari așteptări, locația a fost peste ce gândisem noi. Am comunicat super bine cu gazda, ne.a ajutat cu sfaturi și recomandări iar apartamentul a fost curat, spațios și am găsit tor ce am avut nevoie ca și utilități. Ni s-a părut uneori puțin răcoare pe timp de noapte, dar exista aer condiționat disponibil și am primit instrucțiuni sa îl folosim în caz de nevoie ||| Todo como en las fotos, limpio, con toallas para cada persona. Dos camas

In [0]:

##ADD BERT SENTIMENT 


import sparknlp
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import UniversalSentenceEncoder, SentimentDLModel
from pyspark.ml import Pipeline
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when

# Start Spark NLP 
spark = sparknlp.start()

# Use hotel_with_sentiment from your previous cell
print(f"Starting BERT processing on {hotel_with_sentiment.count():,} hotels")

# ---- BERT SENTIMENT PIPELINE ----
document_assembler = DocumentAssembler() \
    .setInputCol("all_reviews_text") \
    .setOutputCol("document") \
    .setCleanupMode("shrink")

use_embeddings = UniversalSentenceEncoder.pretrained("tfhub_use", "en") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence_embeddings")

sentiment = SentimentDLModel.pretrained("sentimentdl_use_twitter", "en") \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("sentiment")

nlp_pipeline = Pipeline(stages=[document_assembler, use_embeddings, sentiment])

# Apply pipeline to your existing processed data
print("Fitting BERT pipeline...")
nlp_model = nlp_pipeline.fit(hotel_with_sentiment)

print("Transforming data...")
hotel_with_bert = nlp_model.transform(hotel_with_sentiment)

# Extract BERT features
hotel_with_bert = hotel_with_bert.withColumn(
    "bert_sentiment_label",
    col("sentiment.result").getItem(0)
).withColumn(
    "bert_sentiment_score",
    when(col("bert_sentiment_label") == "positive", 1.0)
    .when(col("bert_sentiment_label") == "negative", 0.0)
    .otherwise(0.5)
).withColumn(
    "bert_score_gap",
    (col("review_score") / 10) - col("bert_sentiment_score")
).withColumn(
    "bert_sentiment_mismatch",
    when(
        (col("review_score") >= 8) & (col("bert_sentiment_label") == "negative"),
        1
    ).otherwise(0)
)

# Drop intermediate Spark NLP columns
hotel_final = hotel_with_bert.drop("document", "sentence_embeddings", "sentiment")


print(f"\n=== Final Column Count: {len(hotel_final.columns)} ===")
print("New BERT columns added:")
print("  - bert_sentiment_label")
print("  - bert_sentiment_score")
print("  - bert_score_gap")
print("  - bert_sentiment_mismatch")

# Quick validation
print("\n=== BERT Sentiment Distribution ===")
display(
    hotel_final
    .filter(col("all_reviews_text").isNotNull())
    .groupBy("bert_sentiment_label")
    .agg(
        F.count("*").alias("count"),
        F.round(F.avg("review_score"), 2).alias("avg_review_score"),
        F.round(F.avg("sentiment_ratio"), 2).alias("avg_keyword_sentiment")
    )
)

Starting BERT processing on 62,643 hotels
tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[OK!]
Fitting BERT pipeline...
🏃 View run likeable-doe-829 at: https://adb-983293358114278.18.azuredatabricks.net/ml/experiments/200762682747142/runs/b0f2cd19d04349458101cd556bb3d64d
🧪 View experiment at: https://adb-983293358114278.18.azuredatabricks.net/ml/experiments/200762682747142
Transforming data...

=== Final Column Count: 146 ===
New BERT columns added:
  - bert_sentiment_label
  - bert_sentiment_score
  - bert_score_gap
  - bert_sentiment_mismatch

=== BERT Sentiment Distribution ===


bert_sentiment_label,count,avg_review_score,avg_keyword_sentiment
positive,20806,8.34,0.5
null,25454,6.53,0.5
neutral,3683,8.03,0.5
negative,12700,7.82,0.5


In [0]:
# ============================================================
# CELL: REFINE SENTIMENT MISMATCH (Normalized Scale)
# ============================================================

# Recalculate using normalized scales (0-1 vs 0-1)
hotel_final = hotel_final.withColumn(
    "normalized_review_score", 
    col("review_score") / 10.0
).withColumn(
    "true_sentiment_gap",
    col("normalized_review_score") - col("bert_sentiment_score")
).withColumn(
    "is_highly_overrated",
    (col("true_sentiment_gap") > 0.3) & (col("num_reviews_available") > 5)
).withColumn(
    "is_hidden_gem",
    (col("true_sentiment_gap") < -0.2) & (col("num_reviews_available") > 5)
)

print("Refined sentiment mismatch logic added.")
display(hotel_final.filter(col("is_highly_overrated") == True).select("hotel_id", "title", "review_score", "bert_sentiment_score", "true_sentiment_gap").limit(5))

Refined sentiment mismatch logic added.


hotel_id,title,review_score,bert_sentiment_score,true_sentiment_gap
188199,Residenza Sciarra B&B,8.0,0.0,0.8
11318279,MONTI QUIET AND SPACIOUS APARTMENT,9.1,0.0,0.9099999999999999
7250866,Casa Vacanze Campo de' Fiori,8.0,0.0,0.8
8914023,Domus Otto,8.8,0.0,0.8800000000000001
48041,Pantheon Suites Rome 2,8.2,0.0,0.82


## Save the data

In [0]:

#  SAVE FINAL DATA

# Save the final dataset with all features (keywords + BERT)
hotel_final.write.mode("overwrite").parquet("dbfs:/FileStore/project/gold/hotel_final_nlp")

print(f"Saved {hotel_final.count():,} hotels with {len(hotel_final.columns)} columns")

# Show sample of key columns for Random Forest
print("\n=== Sample for Random Forest ===")
display(
    hotel_final
    .filter(col("all_reviews_text").isNotNull())
    .select(
        "hotel_id", "city", "review_score",
        # Your existing keyword features
        "total_complaints", "total_praises", "sentiment_ratio", "complaint_ratio",
        "sentiment_polarity", "sentiment_gap",
        # Gap signals
        "noise_gap_signal", "cleanliness_gap_signal", "location_gap_signal",
        # New BERT features
        "bert_sentiment_score", "bert_score_gap", "bert_sentiment_mismatch"
    )
    .limit(20)
)

Saved 62,643 hotels with 150 columns

=== Sample for Random Forest ===


hotel_id,city,review_score,total_complaints,total_praises,sentiment_ratio,complaint_ratio,sentiment_polarity,sentiment_gap,noise_gap_signal,cleanliness_gap_signal,location_gap_signal,bert_sentiment_score,bert_score_gap,bert_sentiment_mismatch
188199,Rome,8.0,-6,-6,0.5,0.0,0.7037037037037037,-0.10370370370370374,0.0,1.0,-0.5,0.0,0.8,1
11318279,Rome,9.1,-6,-6,0.5,0.0,1.0,-0.18000000000000005,0.0,0.0,0.0,0.0,0.9099999999999999,1
8591596,Rome,9.2,-6,-6,0.5,0.0,0.9583333333333334,-0.11833333333333351,0.0,1.0,0.0,1.0,-0.08000000000000007,0
10963876,Rome,8.2,-6,-6,0.5,0.0,0.0,0.6399999999999999,0.0,0.0,-0.5,1.0,-0.18000000000000005,0
7250866,Rome,8.0,-6,-6,0.5,0.0,0.696969696969697,-0.09696969696969704,0.0,1.0,-0.5,0.0,0.8,1
11549337,Rome,8.7,-6,-6,0.5,0.0,0.8,-0.060000000000000164,0.0,-0.5,-0.5,1.0,-0.13000000000000012,0
12950226,Rome,9.6,-6,-6,0.5,0.0,0.0,0.9199999999999999,0.0,0.0,0.0,1.0,-0.040000000000000036,0
12086299,Rome,null,-6,-6,0.5,0.0,1.0,null,-0.5,0.0,0.0,0.0,null,0
8914023,Rome,8.8,-6,-6,0.5,0.0,1.0,-0.23999999999999988,0.0,1.0,-0.5,0.0,0.8800000000000001,1
2078791,Rome,8.2,-6,-6,0.5,0.0,0.75,-0.1100000000000001,0.0,1.0,0.0,1.0,-0.18000000000000005,0


#Viz


In [0]:

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# For Databricks - convert Spark DF to Pandas for plotting
# Load the final enriched dataset
hotel_final = spark.read.parquet("dbfs:/FileStore/project/gold/hotel_final_nlp")
df = hotel_final.toPandas()
print(f"Loaded {len(df):,} hotels")



Loaded 62,643 hotels


## Spatial + geographical 

In [0]:


# Sample data for performance (adjust sample size as needed)
df_map = df[df['lat'].notna() & df['lon'].notna()].copy()
df_map_sample = df_map.sample(n=min(5000, len(df_map)), random_state=42)

fig = px.scatter_mapbox(
    df_map_sample,
    lat='lat',
    lon='lon',
    color='review_score',
    size='total_osm_pois_1km' if 'total_osm_pois_1km' in df_map_sample.columns else None,
    color_continuous_scale='RdYlGn',
    hover_name='title' if 'title' in df_map_sample.columns else None,
    hover_data=['city', 'review_score', 'nightlife_count_500m'] if 'nightlife_count_500m' in df_map_sample.columns else ['city', 'review_score'],
    zoom=2,
    height=600,
    title='Hotels by Review Score (Color) and POI Density (Size)'
)

fig.update_layout(
    mapbox_style='carto-positron',
    margin={'r': 0, 't': 50, 'l': 0, 'b': 0}
)
fig.show()


In [0]:


# Select a city (change as needed)
CITY = 'Rome'
df_city = df[df['city'] == CITY].copy()
print(f"Hotels in {CITY}: {len(df_city):,}")

# Create figure with hotels colored by noise score
noise_col = 'avg_noise_score_500m' if 'avg_noise_score_500m' in df_city.columns else 'review_score'

fig = px.scatter_mapbox(
    df_city,
    lat='lat',
    lon='lon',
    color=noise_col,
    color_continuous_scale='Reds',
    hover_name='title' if 'title' in df_city.columns else None,
    hover_data=['review_score', 'nightlife_count_500m', 'noise_sources_500m'] if 'noise_sources_500m' in df_city.columns else ['review_score'],
    zoom=11,
    height=600,
    title=f'{CITY}: Hotels Colored by Noise Score'
)

fig.update_layout(
    mapbox_style='carto-positron',
    margin={'r': 0, 't': 50, 'l': 0, 'b': 0}
)

fig.show()

Hotels in Rome: 16,883


In [0]:


# Geohash encoding function (recreated since not in final dataset)
def encode_geohash(lat, lon, precision=6):
    """Encode lat/lon to geohash string"""
    if lat is None or lon is None:
        return None
    
    try:
        lat = float(lat)
        lon = float(lon)
    except (ValueError, TypeError):
        return None
    
    if lat < -90 or lat > 90 or lon < -180 or lon > 180:
        return None
    
    BASE32 = '0123456789bcdefghjkmnpqrstuvwxyz'
    lat_range = [-90.0, 90.0]
    lon_range = [-180.0, 180.0]
    
    geohash = []
    bits = [16, 8, 4, 2, 1]
    bit = 0
    ch = 0
    even = True
    
    while len(geohash) < precision:
        if even:
            mid = (lon_range[0] + lon_range[1]) / 2
            if lon > mid:
                ch |= bits[bit]
                lon_range[0] = mid
            else:
                lon_range[1] = mid
        else:
            mid = (lat_range[0] + lat_range[1]) / 2
            if lat > mid:
                ch |= bits[bit]
                lat_range[0] = mid
            else:
                lat_range[1] = mid
        
        even = not even
        if bit < 4:
            bit += 1
        else:
            geohash.append(BASE32[ch])
            bit = 0
            ch = 0
    
    return ''.join(geohash)

# Filter to a specific city
CITY = 'London'
df_city = df[df['city'] == CITY].copy()
print(f"Hotels in {CITY}: {len(df_city):,}")

# Compute geohash for each hotel
df_city['geohash'] = df_city.apply(lambda row: encode_geohash(row['lat'], row['lon'], precision=6), axis=1)

# Aggregate by geohash
geohash_counts = df_city.groupby('geohash').agg({
    'hotel_id': 'count',
    'lat': 'mean',
    'lon': 'mean',
    'review_score': 'mean'
}).reset_index()
geohash_counts.columns = ['geohash', 'hotel_count', 'lat', 'lon', 'avg_score']

print(f"Unique geohash cells: {len(geohash_counts)}")

fig = px.scatter_mapbox(
    geohash_counts,
    lat='lat',
    lon='lon',
    size='hotel_count',
    color='avg_score',
    color_continuous_scale='Viridis',
    hover_data=['geohash', 'hotel_count', 'avg_score'],
    zoom=10,
    height=600,
    title=f'{CITY}: Geohash Cells by Hotel Density and Avg Score'
)

fig.update_layout(
    mapbox_style='carto-positron',
    margin={'r': 0, 't': 50, 'l': 0, 'b': 0}
)

fig.show()

Hotels in London: 15,556
Unique geohash cells: 1092


## Noise Analysis

In [0]:


if 'claims_quiet' in df.columns and 'complaint_noise' in df.columns:
    # Aggregate by city
    noise_gap_df = df.groupby('city').agg({
        'claims_quiet': 'sum',
        'complaint_noise': 'sum',
        'hotel_id': 'count'
    }).reset_index()
    noise_gap_df.columns = ['city', 'claims_quiet', 'noise_complaints', 'total_hotels']
    
    # Calculate percentages
    noise_gap_df['pct_claims_quiet'] = (noise_gap_df['claims_quiet'] / noise_gap_df['total_hotels'] * 100).round(1)
    noise_gap_df['pct_noise_complaints'] = (noise_gap_df['noise_complaints'] / noise_gap_df['total_hotels'] * 100).round(1)
    
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=noise_gap_df['city'],
        y=noise_gap_df['pct_claims_quiet'],
        name='Claims "Quiet" (%)',
        marker_color='lightgreen'
    ))
    
    fig.add_trace(go.Bar(
        x=noise_gap_df['city'],
        y=noise_gap_df['pct_noise_complaints'],
        name='Has Noise Complaints (%)',
        marker_color='salmon'
    ))
    
    fig.update_layout(
        title='Hotels Claiming "Quiet" vs Hotels with Noise Complaints by City',
        xaxis_title='City',
        yaxis_title='Percentage of Hotels',
        barmode='group',
        height=500,
        legend=dict(yanchor='top', y=0.99, xanchor='right', x=0.99)
    )
    
    fig.show()
else:
    print("Required columns not found")

## NLP & Sentiment Visualizations

In [0]:


complaint_cols = ['complaint_noise', 'complaint_cleanliness', 'complaint_location']
available_complaints = [c for c in complaint_cols if c in df.columns]

if len(available_complaints) > 0:
    # Calculate complaint rates by city
    complaint_rates = df.groupby('city')[available_complaints].mean() * 100
    complaint_rates = complaint_rates.reset_index()
    
    # Melt for plotting
    complaint_melted = complaint_rates.melt(
        id_vars='city',
        value_vars=available_complaints,
        var_name='complaint_type',
        value_name='rate'
    )
    
    # Clean up labels
    complaint_melted['complaint_type'] = complaint_melted['complaint_type'].str.replace('complaint_', '').str.title()
    
    fig = px.bar(
        complaint_melted,
        x='city',
        y='rate',
        color='complaint_type',
        title='Complaint Types by City (% of Hotels)',
        labels={'rate': 'Percentage of Hotels', 'city': 'City', 'complaint_type': 'Complaint Type'},
        barmode='group',
        height=500,
        color_discrete_sequence=['#FF6B6B', '#4ECDC4', '#45B7D1']
    )
    
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()
else:
    print("No complaint columns found")